# Lab 3.2: Wide EP Production Deployment

## 🎯 Overview

Welcome to Lab 3.2! Now that you understand Expert Parallelism concepts from Lab 3.1, you'll **deploy a real MoE model** (DeepSeek-R1) using Wide EP in a production Kubernetes environment.

**What you'll do in this lab:**
1. ✅ Verify your environment is ready
2. 🔧 Build a custom Docker image with DeepEP support
3. 🚀 Deploy DeepSeek-R1 with Wide Expert Parallelism
4. 📊 Monitor expert load balancing (EPLB) in action
5. 🎯 Benchmark and optimize performance

**Time Required**: 90-120 minutes (includes Docker image build)

**Hardware Requirements**: 
- 16 GPUs minimum (2 nodes × 8 GPUs each)
- High-bandwidth interconnect (NVLink or InfiniBand recommended)

---

## ⚠️ Before You Start

**Prerequisites (must be completed first):**
- ✅ Lab 3.1: Expert Parallelism Foundations
- ✅ Kubernetes cluster with GPU nodes
- ✅ **Dynamo Operator already installed** (from Lab 1 or Lab 2)
- ✅ kubectl configured and working
- ✅ HuggingFace account with access token

💡 **Note**: This lab assumes you've completed Lab 1 or Lab 2, where you installed the Dynamo Operator. We'll skip the operator installation and focus on deploying the MoE model.

---

## 📋 Table of Contents

**Part 1: Setup & Prerequisites**
- [Quick Recap: Lab 3.1 Concepts](#Quick-Recap:-Lab-3.1-Concepts)
- [Prerequisites Check](#Prerequisites-Check)

**Part 2: Deployment**
- [Understanding Your Deployment Options](#Understanding-Your-Deployment-Options)
- [Step-by-Step Deployment Guide](#Step-by-Step-Deployment-Guide)

**Part 3: Configuration Deep Dive**
- [SGLang Configuration Details](#Section-3:-Deploying-MoE-Models-with-SGLang-and-Expert-Parallelism)
- [Monitoring Expert Parallelism and EPLB](#Monitoring-Expert-Parallelism-and-EPLB)

**Part 4: Performance**
- [Benchmarking Your Deployment](#Section-4:-Performance-Benchmarking-for-EP-Deployments)

**Wrap-Up**
- [Summary](#Summary)

---

## 🔄 Quick Recap: Lab 3.1 Concepts

In Lab 3.1, you learned the foundations of Expert Parallelism. Here's a quick refresher:

**Key Concepts**:
- **MoE Models**: Only activate a subset of experts per token (e.g., 8 out of 256 experts)
- **Expert Parallelism (EP)**: Distribute experts across GPUs to scale capacity
- **Wide EP**: Spread experts across many nodes for maximum throughput
- **EPLB**: Dynamic load balancing to prevent GPU hotspots

**What you're deploying today:**
```
DeepSeek-R1 Model:
  - 671B total parameters
  - 256 experts (distributed via EP)
  - Only 8 experts active per token (~37B active params)
  - Disaggregated: Separate prefill & decode workers
```

**Deployment Architecture** (what we're building):
```
                ┌─────────────┐
                │  Frontend   │
                │  (CPU)      │
                └──────┬──────┘
                       │
           ┌───────────┴───────────┐
           │                       │
    ┌──────▼────────┐       ┌──────▼────────┐
    │ Prefill Worker│       │ Decode Worker │
    │   Node 1      │       │   Node 2      │
    │   8 GPUs      │──────▶│   8 GPUs      │
    │               │ NIXL  │               │
    │  TP=8, EP=8   │  KV   │ TP=8, DP=8    │
    │               │       │ EP=8          │
    └───────────────┘       └───────────────┘
```

Now let's verify your environment and deploy!

---

## ✅ Prerequisites Check

Before we start deploying, let's verify your environment is ready. Run the following checks:

---

### Check 1: Verify Kubernetes Access

Make sure you can access your Kubernetes cluster and see your GPU nodes.


### Check 2: Verify GPU Availability

Check that you have at least 16 GPUs available across your nodes:


In [1]:
%%bash
# Check Kubernetes access and nodes
echo "=== Kubernetes Cluster Info ==="
kubectl cluster-info

echo -e "\n=== GPU Nodes ==="
kubectl get nodes -o custom-columns=NAME:.metadata.name,GPUs:.status.allocatable.'nvidia\.com/gpu'

echo -e "\n✅ If you see your nodes with GPUs listed above, you're good to go!"

=== Kubernetes Cluster Info ===
Kubernetes control plane is running at https://pu.mk8scluster-e00wp16ma54aq8nxch.mk8s.eu-north1.nebius.cloud:443
CoreDNS is running at https://pu.mk8scluster-e00wp16ma54aq8nxch.mk8s.eu-north1.nebius.cloud:443/api/v1/namespaces/kube-system/services/coredns:udp-53/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.

=== GPU Nodes ===
NAME                                 GPUs
computeinstance-e00a2atjrzyqw4exyb   8
computeinstance-e00agtdhe3kkf3n9bm   8
computeinstance-e00b2fejeckmrtrapb   8
computeinstance-e00bqd1zygs98b1a4v   <none>
computeinstance-e00bt38gd8kqc4cv87   8
computeinstance-e00etcykgsd2pc3jeg   8
computeinstance-e00fnwgf81feck707q   8
computeinstance-e00g0nhq7x94mnke6q   8
computeinstance-e00gnc7w1wke8558zb   8
computeinstance-e00j0zkrwwx8g4fwwm   8
computeinstance-e00k0jh433ns6pjwpv   8
computeinstance-e00m1pty82yq5cdbz4   8
computeinstance-e00mvtw5qrc0ckppvb   <none>
computeinstance-e00nq20bp314y9yqx1   8


### Check 3: Verify Dynamo Operator is Installed

Since you completed Lab 1 or Lab 2, the Dynamo Operator should already be installed. Let's verify:


In [3]:
%%bash
export NAMESPACE="dynamo-workshop"

echo "=== Checking Dynamo CRDs ==="
kubectl get crd | grep dynamo || echo "⚠️  No Dynamo CRDs found!"

echo -e "\n=== Checking Dynamo Operator Pod ==="
kubectl get pods -n ${NAMESPACE} -l app.kubernetes.io/name=dynamo-operator || echo "⚠️  Dynamo operator not found!"

echo -e "\n=== Checking Platform Components ==="
kubectl get pods -n ${NAMESPACE}

echo -e "\n✅ If you see dynamo-operator, etcd, and nats pods Running above, you're ready!"
echo "⚠️  If not, please complete Lab 1 or Lab 2 first to install the Dynamo Operator."

=== Checking Dynamo CRDs ===
dynamocomponentdeployments.nvidia.com               2025-10-21T14:49:52Z
dynamocomponents.nvidia.com                         2025-10-25T05:16:10Z
dynamographdeploymentrequests.nvidia.com            2025-10-13T17:52:51Z
dynamographdeployments.nvidia.com                   2025-09-04T20:56:40Z

=== Checking Dynamo Operator Pod ===
NAME                                                              READY   STATUS    RESTARTS   AGE
dynamo-platform-dynamo-operator-controller-manager-79d445cnfj94   2/2     Running   0          33h

=== Checking Platform Components ===
NAME                                                              READY   STATUS    RESTARTS   AGE
dynamo-platform-dynamo-operator-controller-manager-79d445cnfj94   2/2     Running   0          33h
dynamo-platform-etcd-0                                            1/1     Running   0          33h
dynamo-platform-nats-0                                            1/1     Running   0          33h

✅ If you

---

## 🚀 Understanding Your Deployment Options

Great! Your environment is ready. Now let's understand what we're about to deploy.

### What is Wide EP?

**Wide Expert Parallelism (Wide EP)** distributes the 256 experts of DeepSeek-R1 across multiple GPUs:

```
Without EP (Won't fit!):          With Wide EP (✅ Works!):
┌─────────────────┐              ┌─────────┬─────────┬─────────┐
│   Single GPU    │              │  GPU 0  │  GPU 1  │  GPU 2  │
│  256 experts    │              │ 32 exp  │ 32 exp  │ 32 exp  │...
│  (Out of memory)│              └─────────┴─────────┴─────────┘
└─────────────────┘              Tokens routed to correct expert
```

### Deployment Configurations Available

We have **two pre-configured deployment options** for 16 GPUs:

**Option 1: Single-Node Workers** ⭐ **RECOMMENDED**
- **Hardware**: 2 nodes × 8 GPUs each = 16 GPUs total
- **Prefill Worker**: 1 pod on Node 1 (8 GPUs: TP=8, EP=8)
- **Decode Worker**: 1 pod on Node 2 (8 GPUs: TP=8, DP=8, EP=8)
- **Benefits**: Simpler, faster NVLink, easier debugging
- **File**: `k8s/deepseek-r1-8gpu-singlenode.yaml`

**Option 2: Multi-Node Workers** (Advanced)
- **Hardware**: 4 nodes × 4 GPUs each = 16 GPUs total
- **Prefill Worker**: 2 pods across 2 nodes (TP=8 cross-node)
- **Decode Worker**: 2 pods across 2 nodes (TP=8 cross-node)
- **Benefits**: Shows multi-node coordination
- **Requirements**: Different hardware (4 nodes instead of 2)
- **File**: `k8s/deepseek-r1-16gpu-multinode.yaml`

💡 **For this lab, we'll use Option 1** (Single-Node Workers) as it's simpler and provides better performance.

---

## 📝 Step-by-Step Deployment Guide

Now let's deploy DeepSeek-R1 with Wide EP! Follow these steps in order.

---

### Step 1: Set Environment Variables

First, set the namespace variable that will be used throughout the deployment.


In [ ]:
%%bash
# Set namespace for all subsequent commands
export NAMESPACE="dynamo-workshop"

echo "🔧 Using namespace: ${NAMESPACE}"
echo "✅ This namespace should already have Dynamo Operator installed from Lab 1/2"

### Step 2: Create HuggingFace Token Secret

Create a Kubernetes secret with your HuggingFace token for downloading the DeepSeek-R1 model.

**Get your token**: https://huggingface.co/settings/tokens (requires accepting DeepSeek-R1 license)


In [ ]:
%%bash
export NAMESPACE="dynamo-workshop"

# Create HuggingFace token secret
# ⚠️ IMPORTANT: Replace 'your_hf_token_here' with your actual HuggingFace token!
# Get your token from: https://huggingface.co/settings/tokens

# Uncomment and edit the line below with your actual token:
# kubectl create secret generic hf-token-secret \
#   --from-literal=HF_TOKEN='your_hf_token_here' \
#   -n ${NAMESPACE}

# Check if secret already exists
kubectl get secret hf-token-secret -n ${NAMESPACE} 2>/dev/null && \
  echo "✅ HuggingFace token secret already exists!" || \
  echo "⚠️  Please create the secret by uncommenting and editing the command above"

### Step 3: Build Custom Docker Image (Part 1 - Overview)

The deployment requires a custom Docker image with **DeepEP** support for Wide EP functionality.

**What is DeepEP?** SGLang's optimized backend for Expert Parallelism with:
- ✅ Efficient all-to-all communication for expert routing
- ✅ EPLB (Expert Parallel Load Balancer) for dynamic load balancing
- ✅ Optimized GroupGEMM kernels for MoE layers

**Two options:**
1. **Build locally** (30-60 mins) - Use for single-node testing
2. **Build and push to registry** - Required for multi-node deployments


In [ ]:
%%bash
# Build the Docker image with DeepEP support
# This will take 30-60 minutes

echo "🏗️  Building Docker image with DeepEP support..."
echo "📍 This may take 30-60 minutes depending on your machine"
echo ""

# Uncomment the following lines to build:
# cd /mnt/raid/dynamo-workshop/dynamo
# git checkout v0.6.0
# docker build -f container/Dockerfile.sglang-wideep -t dynamo-wideep:0.6.0 .

echo "⚠️  Uncomment the build commands above to start the build"
echo "💡 Or use a pre-built image if available in your registry"

### Step 3 (Part 2): Push to Registry (Optional - for Multi-Node only)

**Only needed if deploying multi-node** where pods run on different physical nodes.

For **single-node deployment** (recommended), you can skip this and use the local image.

**For multi-node**, tag and push to your container registry:


In [ ]:
%%bash
# OPTIONAL: Only for multi-node deployments
# Tag and push image to your container registry

# Replace <your-registry> with your actual registry (e.g., docker.io/username, gcr.io/project)
# Uncomment to push:

# docker tag dynamo-wideep:0.6.0 <your-registry>/dynamo-wideep:0.6.0
# docker push <your-registry>/dynamo-wideep:0.6.0

# Then update manifests:
# cd /mnt/raid/dynamo-workshop/lab3
# sed -i 's|<your-registry>/dynamo-wideep:0.6.0|your-actual-registry/dynamo-wideep:0.6.0|g' k8s/deepseek-r1-8gpu-singlenode.yaml

echo "💡 For single-node deployment, you can use the local image: dynamo-wideep:0.6.0"
echo "💡 For multi-node, uncomment and run the commands above"


---

### Step 4: Deploy DeepSeek-R1 with Wide EP! 🚀

Time to deploy! This will create a **disaggregated deployment** with:
- **Frontend**: Handles requests (CPU only)
- **Prefill Worker**: Processes prompts (8 GPUs on Node 1)
- **Decode Worker**: Generates tokens (8 GPUs on Node 2)
- **Expert Parallelism**: 256 experts distributed across 8 GPUs per worker
- **EPLB**: Dynamic load balancing of experts


In [4]:
%%bash
export NAMESPACE="dynamo-workshop"

echo "🚀 Deploying DeepSeek-R1 with Wide Expert Parallelism..."
echo ""

# Deploy using Single-Node Workers configuration (recommended)
kubectl apply -f k8s/deepseek-r1-8gpu-singlenode.yaml -n ${NAMESPACE}

# For multi-node deployment (if you have 4 nodes × 4 GPUs), use:
# kubectl apply -f k8s/deepseek-r1-16gpu-multinode.yaml -n ${NAMESPACE}

echo ""
echo "✅ Deployment created!"
echo ""
echo "📊 Checking deployment status..."
kubectl get dynamographdeployment -n ${NAMESPACE}

echo ""
echo "💡 The pods will now start. This may take 5-10 minutes as:"
echo "   1. Model weights are downloaded from HuggingFace (~100GB)"
echo "   2. Workers initialize and load the model into GPU memory"
echo "   3. Expert Parallelism topology is established"

🚀 Deploying DeepSeek-R1 with Wide Expert Parallelism...

dynamographdeployment.nvidia.com/sgl-dsr1-8gpu created

✅ Deployment created!

📊 Checking deployment status...
NAME            AGE
sgl-dsr1-8gpu   1s

💡 The pods will now start. This may take 5-10 minutes as:
   1. Model weights are downloaded from HuggingFace (~100GB)
   2. Workers initialize and load the model into GPU memory
   3. Expert Parallelism topology is established


### Step 5: Monitor Deployment Progress

Watch the pods as they start up. You should see:
1. **Frontend pod** (starts quickly, CPU only)
2. **Prefill worker pod(s)** (downloading model, loading to GPU)
3. **Decode worker pod(s)** (downloading model, loading to GPU)

**Expected pod states:**
- `Pending` → `ContainerCreating` → `Running`
- Model download takes 5-10 minutes
- Look for "Model loaded successfully" in logs

💡 **Press Ctrl+C to stop watching** (pods continue running in background)


In [ ]:
%%bash
export NAMESPACE="dynamo-workshop"

# Watch pods being created and starting
# Press Ctrl+C to stop watching
kubectl get pods -n ${NAMESPACE} -w

### Step 6: Check Pod Logs (Troubleshooting)

If pods are stuck or you want to see progress, check the logs:

**What to look for:**
- ✅ "Downloading model..." → "Model loaded successfully"
- ✅ "EP topology established" → Expert Parallelism is working
- ✅ "EPLB initialized" → Load balancing is active
- ⚠️ Errors about memory, CUDA, or missing files


In [ ]:
%%bash
export NAMESPACE="dynamo-workshop"

# Check logs of prefill workers
kubectl logs -n ${NAMESPACE} -l component=prefill --tail=50

# Check logs of decode workers
kubectl logs -n ${NAMESPACE} -l component=decode --tail=50

# Check frontend logs
kubectl logs -n ${NAMESPACE} -l component=frontend --tail=50


---

### Step 7: Test the Deployment! 🎉

Once all pods are `Running`, let's test the deployment!

#### Step 7a: Port Forward to Access the Frontend

Create a port forward to access the deployment from your local machine.

💡 **Note**: Keep this running in the background or in a separate terminal while testing.


In [ ]:
%%bash
export NAMESPACE="dynamo-workshop"

echo "🔌 Creating port forward to access the frontend..."
echo "💡 Press Ctrl+C to stop (or run in background with & at the end)"
echo ""

# Port forward to access the frontend
kubectl port-forward svc/deepseek-r1-wideep-frontend 8000:8000 -n ${NAMESPACE}

# To run in background instead:
# kubectl port-forward svc/deepseek-r1-wideep-frontend 8000:8000 -n ${NAMESPACE} &


#### Step 7b: Send a Test Request

Send a test request to verify Expert Parallelism is working!

💡 **Make sure** the port-forward from the previous cell is still running.


In [ ]:
%%bash
echo "🧪 Testing Wide EP deployment with a sample request..."
echo ""

# Test the deployment with a simple request
curl -s http://localhost:8000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "deepseek-ai/DeepSeek-R1",
    "messages": [{"role": "user", "content": "Explain MoE models in one sentence"}],
    "max_tokens": 100
  }' | python3 -m json.tool

echo ""
echo "✅ If you see a response above, your Wide EP deployment is working!"
echo "🎯 Behind the scenes:"
echo "   - Your request was processed by the frontend"
echo "   - Routed to prefill worker (prompt encoding)"
echo "   - Tokens were routed to correct experts (out of 256)"
echo "   - Decode worker generated the response"
echo "   - EPLB balanced the load across GPUs!"


---

## 🎉 Congratulations! Your Wide EP Deployment is Running!

If you received a response above, you've successfully deployed DeepSeek-R1 with Wide Expert Parallelism!

### What You Just Accomplished

✅ **Deployed a 671B parameter MoE model** with 256 experts  
✅ **Configured Expert Parallelism** to distribute experts across GPUs  
✅ **Set up disaggregated serving** with separate prefill/decode workers  
✅ **Enabled EPLB** for dynamic load balancing  
✅ **Made your first inference request** through the Wide EP deployment

### What's Happening Under the Hood?

Every request you send triggers this flow:
1. **Frontend** receives request → routes to prefill worker
2. **Prefill Worker** (8 GPUs):
   - Processes prompt tokens in parallel
   - **Router** selects top-8 experts per token (out of 256)
   - Tokens routed to correct GPUs via all-to-all communication
   - **EPLB** balances load if some experts are hot
   - Generates KV cache
3. **KV Transfer** via NIXL to decode worker
4. **Decode Worker** (8 GPUs):
   - Generates tokens autoregressively
   - Same expert routing and EPLB
5. **Response** streamed back to you

### Next Steps

**Want to learn more?**
- 📚 **Section 3**: Deep dive into SGLang configuration and EPLB tuning
- 📊 **Section 4**: Benchmark and optimize performance
- 🔍 **Explore**: Check out the configuration files in `k8s/` directory

**Ready for production?**
- Scale to more replicas for higher throughput
- Tune EPLB parameters for your workload
- Set up monitoring and alerting

### Cleanup (when done)

To remove the deployment and free up GPUs:

```bash
# For single-node deployment
kubectl delete -f k8s/deepseek-r1-8gpu-singlenode.yaml -n dynamo-workshop

# For multi-node deployment
# kubectl delete -f k8s/deepseek-r1-16gpu-multinode.yaml -n dynamo-workshop
```

---


## 📚 Part 3: Configuration Deep Dive

Now that you have a working deployment, let's understand the configuration details and how to tune it for your needs.

### Understanding the Kubernetes Manifest

The `k8s/deepseek-r1-8gpu-singlenode.yaml` file you deployed contains:

**Key Configuration Parameters:**
- **Expert Parallelism**: `--ep-size 8` (256 experts across 8 GPUs = 32 experts per GPU)
- **Tensor Parallelism**: `--tp-size 8` (model weights split across 8 GPUs)
- **Data Parallelism**: `--dp-size 8` (decode only, for batch processing)
- **EPLB**: `--ep-num-redundant-experts 32` (hot expert replication)
- **Disaggregation**: Separate prefill and decode workers with NIXL transfer

**Why these numbers?**
- DeepSeek-R1 has 256 experts → with EP=8, each GPU handles 32 experts
- TP=8 allows the 37B active parameters to fit across 8 GPUs
- DP=8 on decode enables parallel processing of multiple requests
- EPLB with 32 redundant experts prevents bottlenecks on hot experts
- High-bandwidth interconnect (InfiniBand or NVLink preferred)

**Check GPU availability**:


### SGLang Deployment Configurations

All SGLang configuration is done via command-line arguments in the Kubernetes manifests.

**Available Manifests** (based on official Dynamo recipes):
1. `k8s/deepseek-r1-8gpu-singlenode.yaml` - 8 GPUs (Example 1)
2. `k8s/deepseek-r1-16gpu-multinode.yaml` - 16 GPUs (Example 2)

Each manifest includes complete configuration for:
- Expert Parallelism parameters (`--tp-size`, `--dp-size`, `--ep-size`)
- EPLB settings (EP redundancy, load balancing)
- Memory optimization (`--mem-fraction-static`)
- Disaggregated prefill/decode workers with NIXL transfer

**Reference**: [Official Dynamo Recipes](https://github.com/ai-dynamo/dynamo/tree/main/recipes/deepseek-r1/sglang-wideep)


In [1]:
# List available Kubernetes manifests
from pathlib import Path

print("=" * 60)
print("Lab 3 DeepSeek-R1 Deployment Manifests")
print("=" * 60)

k8s_dir = Path("k8s")
if k8s_dir.exists():
    manifests = [
        ("deepseek-r1-8gpu-singlenode.yaml", "Example 1: 8 GPUs (Single-Node)"),
        ("deepseek-r1-16gpu-multinode.yaml", "Example 2: 16 GPUs (Multi-Node)"),
        ("README.md", "Kubernetes Deployment Guide")
    ]
    
    print("\nAvailable Manifests:")
    print("-" * 60)
    for filename, description in manifests:
        file = k8s_dir / filename
        if file.exists():
            if filename.endswith('.yaml'):
                size = file.stat().st_size
                print(f"  ✓ {filename:<40} ({size:>6,} bytes)")
                print(f"    {description}")
            else:
                print(f"  ✓ {filename:<40} {description}")
else:
    print("  ⚠️  k8s/ directory not found")

print("\n📝 Based on official Dynamo recipes:")
print("   https://github.com/ai-dynamo/dynamo/tree/main/recipes/deepseek-r1/sglang-wideep")
print("=" * 60)


Lab 3 DeepSeek-R1 Deployment Manifests

Available Manifests:
------------------------------------------------------------
  ✓ deepseek-r1-8gpu-singlenode.yaml         ( 3,178 bytes)
    Example 1: 8 GPUs (Single-Node)
  ✓ deepseek-r1-16gpu-multinode.yaml         ( 3,646 bytes)
    Example 2: 16 GPUs (Multi-Node)
  ✓ README.md                                Kubernetes Deployment Guide

📝 Based on official Dynamo recipes:
   https://github.com/ai-dynamo/dynamo/tree/main/recipes/deepseek-r1/sglang-wideep


### Example 1: Single-Node Workers (Recommended for 2 Nodes × 8 GPUs)

**✅ Use this configuration for your setup!** (2 nodes × 8 GPUs each = 16 GPUs total)

This deploys DeepSeek-R1 with disaggregated prefill/decode workers, each worker running on a single node.

**Manifest**: `k8s/deepseek-r1-8gpu-singlenode.yaml`

**Architecture**:
```
                ┌─────────────┐
                │  Frontend   │
                │  (CPU only) │
                └──────┬──────┘
                       │
           ┌───────────┴───────────┐
           │                       │
    ┌──────▼────────┐       ┌──────▼────────┐
    │ Prefill Worker│       │ Decode Worker │
    │   Node 1      │       │   Node 2      │
    │   8 GPUs      │──────▶│   8 GPUs      │
    │               │ NIXL  │               │
    │  TP=8, EP=8   │  KV   │ TP=8, DP=8    │
    │               │       │ EP=8, DP-Attn │
    └───────────────┘       └───────────────┘
```

**Configuration Details**:

**Total Resources**: 16 GPUs across 2 nodes

**Prefill Worker** (1 pod, 8 GPUs on Node 1):
- TP=8: Model tensor parallelism across 8 GPUs (NVLink within node)
- EP=8: 256 experts distributed across 8 GPUs
- Processes prompt encoding
- Transfers KV cache to decode via NIXL

**Decode Worker** (1 pod, 8 GPUs on Node 2):
- TP=8: Model tensor parallelism across 8 GPUs (NVLink within node)
- DP=8: Data parallelism for batch processing
- EP=8: Expert parallelism (256 experts distributed)
- DP Attention: Parallel attention computation
- Receives KV cache from prefill
- Generates tokens autoregressively

**Key Parameters**:
```bash
# Prefill Worker (Node 1)
--model-path deepseek-ai/DeepSeek-R1
--tp-size 8
--ep-size 8
--disaggregation-mode prefill
--disaggregation-transfer-backend nixl

# Decode Worker (Node 2)
--model-path deepseek-ai/DeepSeek-R1
--tp-size 8
--dp-size 8
--ep-size 8
--enable-dp-attention
--disaggregation-mode decode
--disaggregation-transfer-backend nixl
```

**Why this configuration?**
- ✅ **Perfect for 2 nodes × 8 GPUs** (your hardware!)
- ✅ Each worker stays on one node - fast NVLink communication
- ✅ No cross-node TP overhead - better performance
- ✅ Simpler to deploy and debug
- ✅ Optimal for learning Wide EP concepts


### Example 2: Multi-Node Workers (For Larger Clusters)

**Note**: This configuration is for clusters with **4+ nodes** and demonstrates advanced multi-node deployment patterns.

**Your Setup**: You have **2 nodes × 8 GPUs = 16 GPUs**. Use **Example 1** (`deepseek-r1-8gpu-singlenode.yaml`).

**Manifest**: `k8s/deepseek-r1-16gpu-multinode.yaml` (requires 4 nodes × 4 GPUs)

**Architecture** (for 4-node clusters):
```
                ┌─────────────┐
                │  Frontend   │
                │  (CPU only) │
                └──────┬──────┘
                       │
           ┌───────────┴─────────────┐
           │                         │
    ┌──────▼────────────┐     ┌──────▼───────────┐
    │ Prefill Worker    │     │ Decode Worker    │
    │  (Multi-node)     │     │  (Multi-node)    │
    │                   │     │                  │
    │ ┌────────────┐    │     │ ┌────────────┐   │
    │ │Node 1      │    │────▶│ │Node 3      │   │
    │ │4 GPUs      │    │NIXL │ │4 GPUs      │   │
    │ └────────────┘    │ KV  │ └────────────┘   │
    │ ┌────────────┐    │     │ ┌────────────┐   │
    │ │Node 2      │    │     │ │Node 4      │   │
    │ │4 GPUs      │    │     │ │4 GPUs      │   │
    │ └────────────┘    │     │ └────────────┘   │
    │                   │     │                  │
    │ Total: 8 GPUs     │     │ Total: 8 GPUs    │
    │ TP=8, EP=8        │     │ TP=8, DP=8, EP=8 │
    └───────────────────┘     └──────────────────┘
```

**Configuration Details**:

**Total Resources**: 16 GPUs across **4 nodes** (4 GPUs per node)

**Prefill Worker** (2 pods × 4 GPUs = 8 GPUs):
- Multi-node: 2 pods across 2 nodes (Node 1 + Node 2)
- TP=8: Model sharded across 8 GPUs (cross-node via NCCL)
- EP=8: 256 experts distributed across 8 GPUs
- Requires InfiniBand/RDMA for efficient cross-node communication

**Decode Worker** (2 pods × 4 GPUs = 8 GPUs):
- Multi-node: 2 pods across 2 nodes (Node 3 + Node 4)
- TP=8: Model sharded across 8 GPUs (cross-node via NCCL)
- DP=8: Data parallelism for batch processing
- EP=8: Expert parallelism

**When to use this**:
- ⚠️ **You need 4 nodes** with 4 GPUs each (not 2 nodes with 8 GPUs each)
- Shows advanced multi-node coordination
- Demonstrates cross-node TP/EP communication
- Requires excellent inter-node networking (25+ Gbps InfiniBand)

**For your 2-node × 8 GPU setup**: Use Example 1 instead!


In [ ]:
# DeepSeek-R1 Deployment Examples
# Choose the right configuration for your hardware

print("""
=================================================================
DeepSeek-R1 Wide EP Deployment Commands
=================================================================

Example 1: Single-Node Workers (✅ USE THIS for 2 nodes × 8 GPUs)
───────────────────────────────────────────────────────────────
Manifest: k8s/deepseek-r1-8gpu-singlenode.yaml
Deploy:   kubectl apply -f k8s/deepseek-r1-8gpu-singlenode.yaml

Hardware: 2 nodes × 8 GPUs each = 16 GPUs total
Architecture:
  - Prefill: 1 pod × 8 GPUs on Node 1 (TP=8, EP=8)
  - Decode:  1 pod × 8 GPUs on Node 2 (TP=8, DP=8, EP=8)
  - Fast NVLink within each node, no cross-node TP overhead


Example 2: Multi-Node Workers (⚠️  Requires 4 nodes × 4 GPUs)
───────────────────────────────────────────────────────────────
Manifest: k8s/deepseek-r1-16gpu-multinode.yaml
Deploy:   kubectl apply -f k8s/deepseek-r1-16gpu-multinode.yaml

Hardware: 4 nodes × 4 GPUs each = 16 GPUs total (DIFFERENT from above!)
Architecture:
  - Prefill: 2 pods × 4 GPUs each (TP=8 cross-node)
  - Decode:  2 pods × 4 GPUs each (TP=8 cross-node, DP=8, EP=8)
  - Requires excellent inter-node networking (InfiniBand/RDMA)
  - Only use if you have 4 nodes with 4 GPUs each


📝 For your 2-node × 8 GPU setup, use Example 1!

=================================================================
""")


### Monitoring Expert Parallelism and EPLB

When running MoE models with EP and EPLB, monitoring is crucial to ensure optimal performance.

#### Key Metrics to Monitor

**1. Expert Usage Distribution**
```python
# SGLang automatically logs expert usage statistics
# Look for logs like:
# "Expert usage: [0.05, 0.12, 0.03, 0.15, ...]"
# These show the fraction of tokens routed to each expert
```

**2. GPU Utilization per Expert**
```bash
# Use nvidia-smi to check GPU utilization
watch -n 1 nvidia-smi

# For detailed metrics, use DCGM:
dcgmi dmon -e 155,156,203,204 -d 1
# 155 = GPU Utilization
# 156 = Memory Utilization
# 203 = Tensor Core Utilization
# 204 = FP16 Activity
```

**3. EPLB Rebalancing Events**
```python
# Enable verbose logging to see EPLB rebalancing
# Set environment variable: DYNAMO_LOG=debug

# Look for logs like:
# "EPLB: Rebalancing experts after 100 iterations"
# "EPLB: Expert 5 replicated to GPU 2 (high usage: 0.25)"
# "EPLB: Expert 17 removed from GPU 3 (low usage: 0.01)"
```

**4. Network Bandwidth (for Multi-Node)**
```bash
# Monitor InfiniBand bandwidth
ibstat

# Monitor network throughput
iftop -i ib0  # Replace ib0 with your IB interface
```

#### Troubleshooting Common Issues

**Issue 1: Uneven GPU Utilization**
```
Symptoms:
- Some GPUs at 100%, others at <50%
- Throughput lower than expected
- Long token generation times

Solution:
- Enable EPLB: --enable-eplb
- Increase redundant experts: --ep-num-redundant-experts 32
- Adjust rebalancing frequency: --eplb-rebalance-num-iterations 50
```

**Issue 2: High Memory Usage**
```
Symptoms:
- OOM errors
- Cannot create redundant experts

Solution:
- Reduce memory fraction: --mem-fraction-static 0.80 (from 0.85)
- Reduce redundant experts: --ep-num-redundant-experts 16
- Disable features: --disable-radix-cache
```

**Issue 3: Slow Expert All-to-All Communication**
```
Symptoms:
- High latency during expert routing
- Low GPU utilization despite balanced load

Solution:
- Use DeepEP backend: --moe-a2a-backend deepep
- Enable two-batch overlap: --enable-two-batch-overlap
- Check network: Ensure InfiniBand is active and configured
```

**Issue 4: EPLB Not Rebalancing**
```
Symptoms:
- No rebalancing logs
- Expert usage remains imbalanced over time

Solution:
- Enable explicit EPLB: --enable-eplb
- Use appropriate recorder mode: --expert-distribution-recorder-mode stat
- Lower rebalance threshold: --eplb-rebalance-num-iterations 50
```

#### Performance Tuning Tips

**1. Optimize Memory Allocation**
```bash
# Start with conservative memory fraction
--mem-fraction-static 0.80

# Gradually increase if no OOM
--mem-fraction-static 0.85

# Monitor with nvidia-smi
```

**2. Tune Redundant Expert Count**
```bash
# Formula: redundant_experts ≈ num_GPUs / 2 to num_GPUs
# For 32 GPUs: try 16-32 redundant experts

# Start low
--ep-num-redundant-experts 16

# Increase if imbalance persists
--ep-num-redundant-experts 32
```

**3. DeepEP Mode Selection**
```bash
# For prefill (focus on throughput)
--deepep-mode normal

# For decode (focus on latency)
--deepep-mode low_latency
```

**4. Batch Size Tuning**
```bash
# For decode, tune CUDA graph batch size
# Larger = better throughput, more memory
--cuda-graph-bs 128

# If OOM, reduce
--cuda-graph-bs 64
```


---

## 📊 Part 4: Performance Benchmarking

Great! You have a working Wide EP deployment. Now let's measure its performance and understand how EPLB improves throughput.

**In this section, you'll:**
- Learn key metrics for MoE deployments
- Benchmark your Wide EP deployment
- Understand expert load balancing effectiveness
- Compare throughput with and without EPLB

### Objectives
- Benchmark Expert Parallelism and EPLB performance
- Compare single-node vs multi-node deployments
- Measure expert load balancing effectiveness
- Analyze throughput and latency characteristics

### Key Metrics for MoE Models

#### 1. **Throughput Metrics**
```python
# Requests per second across all replicas
# Tokens per second (both input and output)
# Expert activations per second
```

#### 2. **Latency Metrics**
```python
# Time to First Token (TTFT)
# Time per Output Token (TPOT)  
# Expert routing latency
# All-to-all communication time
```

#### 3. **Load Balancing Metrics**
```python
# GPU utilization variance (should be low with EPLB)
# Expert usage distribution (should be balanced)
# EPLB rebalancing frequency
# Redundant expert utilization
```

#### 4. **Resource Utilization**
```python
# GPU memory usage per worker
# Network bandwidth (especially for multi-node)
# CPU usage for pre/post-processing
```

### Benchmarking Exercise 1: Expert Load Distribution

**Goal**: Measure how EPLB improves expert load balancing

**Setup**:
1. Deploy a MoE model WITHOUT EPLB
2. Run workload and measure GPU utilization variance
3. Enable EPLB and re-run same workload
4. Compare results


In [ ]:
import time
import requests
import statistics

def benchmark_deployment(endpoint, num_requests=10):
    """Benchmark an EP deployment"""
    print(f"Benchmarking {endpoint}...")
    print(f"Sending {num_requests} requests...\n")
    
    latencies = []
    
    for i in range(num_requests):
        start = time.time()
        try:
            response = requests.post(
                f"{endpoint}/v1/chat/completions",
                json={
                    "model": "deepseek-ai/DeepSeek-R1",
                    "messages": [{"role": "user", "content": "Hello"}],
                    "max_tokens": 50
                },
                timeout=30
            )
            latency = time.time() - start
            latencies.append(latency)
            print(f"Request {i+1}: {latency:.2f}s")
        except Exception as e:
            print(f"Request {i+1}: Failed - {e}")
    
    if latencies:
        print(f"\nResults:")
        print(f"  Mean latency: {statistics.mean(latencies):.2f}s")
        print(f"  Median latency: {statistics.median(latencies):.2f}s")
        print(f"  Throughput: {num_requests / sum(latencies):.2f} req/s")

# Example usage (uncomment when deployment is running):
# benchmark_deployment("http://localhost:8000", num_requests=10)


## Summary

### What You Learned
- ✅ Wide EP deployments across multiple nodes with SGLang
- ✅ Expert Parallelism configuration and EPLB tuning
- ✅ Advanced performance measurement and optimization
- ✅ Production deployment best practices with Kubernetes
- ✅ Building custom Docker images with DeepEP support

### Key Takeaways
- Wide EP enables datacenter-scale MoE deployments
- EPLB significantly improves load balancing and throughput
- Multi-node deployments require careful network and resource planning
- Custom Docker images are required for DeepEP backend support
- SGLang provides flexible deployment and easier experimentation

### Performance Improvements with Wide EP
Key benefits you can expect:
- **SGLang with DeepEP**: Optimized all-to-all communication for expert routing
- **EPLB**: Balanced GPU utilization, preventing hotspots
- **Multi-Node**: Horizontal scaling with proper network configuration
- **Wide EP**: Better resource utilization across large GPU clusters
- **Disaggregated Serving**: Separate prefill and decode workers for efficiency

### Next Steps
- Apply these techniques to your production deployments
- Experiment with different configurations for your specific workloads
- Contribute optimizations back to the Dynamo community
- Explore the latest features in the [Dynamo repository](https://github.com/ai-dynamo/dynamo)

---

## Congratulations!

You've completed the Dynamo Workshop. You now have the knowledge to:
- Deploy Dynamo from local to datacenter scale
- Choose the right topology for your use case
- Optimize performance with Wide EP and EPLB
- Operate production-grade LLM inference infrastructure
